<a href="https://colab.research.google.com/github/Akshayc10/Data-Science-Portfolio/blob/master/Web%20scrapping%20(Amazon_Books).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install beautifulsoup4

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [ ]:
no_pages = 2
def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/')
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [ ]:
results = []


In [ ]:
for i in range(1, no_pages+1):
    results.append(get_data(i))

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [ ]:
df = pd.read_csv("amazon_products.csv")

In [ ]:
df.shape

(100, 5)

In [ ]:
df.head(100)

,Book Name,Author,Rating,Customers_Rated,Price
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.5 out of 5 stars,"2,467",₹ 395.00
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"4,369",₹ 499.00
2,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6 out of 5 stars,"2,136",₹ 543.00
3,The Power of your Subconscious Mind,Joseph Murphy,4.5 out of 5 stars,"15,776",₹ 159.00
4,A Brief History of Modern India (2019-2020 Edi...,Spectrum Books Pvt.Ltd.,4.5 out of 5 stars,"1,552",₹ 256.00
...,...,...,...,...,...
95,Harry Potter Box Set: The Complete Collection ...,J.K. Rowling,4.6 out of 5 stars,"6,624","₹ 2,550.00"
96,General Knowledge 2021,Manohar Pandey,4.2 out of 5 stars,364,₹ 30.00
97,As a Man Thinketh,James Allen,4.4 out of 5 stars,"3,220",₹ 50.00
98,Death; An Inside Story: A book for all those w...,Sadhguru,4.7 out of 5 stars,"1,341",₹ 263.00


In [ ]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])

In [ ]:
df['Rating'] = pd.to_numeric(df['Rating'])

In [ ]:
df["Price"] = df["Price"].str.replace('₹', '')

In [ ]:
df["Price"] = df["Price"].str.replace(',', '')

In [ ]:
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])

In [ ]:
df['Price'] = df['Price'].astype(int)

In [ ]:
df["Customers_Rated"] = df["Customers_Rated"].str.replace(',', '')

In [ ]:
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')

In [ ]:
df.head(100)

,Book Name,Author,Rating,Customers_Rated,Price
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.5,2467,395
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,4369,499
2,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,2136,543
3,The Power of your Subconscious Mind,Joseph Murphy,4.5,15776,159
4,A Brief History of Modern India (2019-2020 Edi...,Spectrum Books Pvt.Ltd.,4.5,1552,256
...,...,...,...,...,...
95,Harry Potter Box Set: The Complete Collection ...,J.K. Rowling,4.6,6624,2550
96,General Knowledge 2021,Manohar Pandey,4.2,364,30
97,As a Man Thinketh,James Allen,4.4,3220,50
98,Death; An Inside Story: A book for all those w...,Sadhguru,4.7,1341,263


In [ ]:
df.dtypes

Book Name           object
Author              object
Rating             float64
Customers_Rated      int64
Price                int64
dtype: object

In [ ]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)

In [ ]:
count_nan = len(df) - df.count()
count_nan

Book Name          0
Author             0
Rating             0
Customers_Rated    0
Price              0
dtype: int64

In [ ]:
df1=df.drop_duplicates(subset=None, keep='first', inplace=False)
df1

,Book Name,Author,Rating,Customers_Rated,Price
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.5,2467,395
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,4369,499
2,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,2136,543
3,The Power of your Subconscious Mind,Joseph Murphy,4.5,15776,159
4,A Brief History of Modern India (2019-2020 Edi...,Spectrum Books Pvt.Ltd.,4.5,1552,256
5,1001 Activities Book,Dreamland Publications,4.4,1265,280
6,101 Panchatantra Stories,Dreamland Publications,4.3,1174,119
7,The Alchemist,Paulo Coelho,4.7,23546,228
8,25 Years UPSC IAS/ IPS Prelims Topic-wise Solv...,Disha Experts,4.3,1653,297
9,Think and Grow Rich,Napoleon Hill,4.5,18172,149


In [ ]:
data = df1.sort_values(["Price"], axis=0, ascending=False)[:15]
data

,Book Name,Author,Rating,Customers_Rated,Price
45,Harry Potter Box Set: The Complete Collection ...,J.K. Rowling,4.6,6624,2550
23,My First Learning Library Box Set: 20 Board Bo...,Wonder House Books,4.6,1007,899
16,Quantitative Aptitude for Competitive Examinat...,R S Aggarwal,4.4,5219,583
2,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,2136,543
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,4369,499
38,The Intelligent Investor (English) Paperback –...,Benjamin Graham,4.4,6804,455
12,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.6,15996,399
21,World’s Greatest Books For Personal Growth & W...,Various,4.6,401,399
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.5,2467,395
49,CBSE All In One Social Science Class 10 for 20...,Farah Sultan Madhumita Pattrea,3.9,32,349


In [ ]:
df1.to_csv('Amazon_Bookss.csv')

Top Rated Books and Authors wrt Customers Rated

In [ ]:
data = df1[df1['Customers_Rated'] > 1000]
data = data.sort_values(['Rating'],axis=0, ascending=False)[:15]
data

,Book Name,Author,Rating,Customers_Rated,Price
44,Bhagavad Gita: Yatharoop (Hindi),A.C. Bhaktivendanta Swami Prabhupada,4.8,2954,189
48,Death; An Inside Story: A book for all those w...,Sadhguru,4.7,1341,263
7,The Alchemist,Paulo Coelho,4.7,23546,228
20,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,8569,268
45,Harry Potter Box Set: The Complete Collection ...,J.K. Rowling,4.6,6624,2550
39,The Monk Who Sold His Ferrari,Robin Sharma,4.6,6606,178
12,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.6,15996,399
23,My First Learning Library Box Set: 20 Board Bo...,Wonder House Books,4.6,1007,899
2,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,2136,543
0,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.5,2467,395


Most Customer Rated Authors and Books

In [ ]:
data = df1.sort_values(["Customers_Rated"], axis=0, ascending=False)[:20]
data

,Book Name,Author,Rating,Customers_Rated,Price
7,The Alchemist,Paulo Coelho,4.7,23546,228
9,Think and Grow Rich,Napoleon Hill,4.5,18172,149
32,How to Win Friends and Influence People,Dale Carnegie,4.5,16692,175
12,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.6,15996,399
3,The Power of your Subconscious Mind,Joseph Murphy,4.5,15776,159
19,Rich Dad Poor Dad : What The Rich Teach Their ...,Robert T. Kiyosaki,4.5,15712,348
35,The Subtle Art of Not Giving a F*ck,Mark Manson,4.4,15559,299
10,Word Power Made Easy,Norman Lewis,4.4,11888,145
29,Man's Search For Meaning: The classic tribute ...,Viktor E Frankl,4.4,9693,260
20,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,8569,268
